In [1]:
print("Hi")

Hi


In [2]:
import tensorflow as tf
print(tf.__version__)


2.13.0


In [3]:
conda list | findstr tensorflow


tensorflow                      2.13.0           pypi_0           pypi
tensorflow-estimator            2.13.0           pypi_0           pypi
tensorflow-intel                2.13.0           pypi_0           pypi
tensorflow-io-gcs-filesystem    0.31.0           pypi_0           pypi

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip list | findstr keras

keras                        2.13.1
Note: you may need to restart the kernel to use updated packages.


In [5]:
tf.keras

<module 'keras.api._v2.keras' from 'c:\\Users\\NIDHI\\anaconda3\\envs\\newenv311\\Lib\\site-packages\\keras\\api\\_v2\\keras\\__init__.py'>

In [6]:
import sys
print(sys.executable)


c:\Users\NIDHI\anaconda3\envs\newenv311\python.exe


In [7]:
!python -m pip show pandas


Name: pandas
Version: 2.3.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

 Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
 All rights reserved.

 Copyright (c) 2011-2023, Open source contributors.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

 * Neither the name of the copyright holder nor the names of its
   contribut

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [9]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
# Preprocess the data
# Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [11]:
# Encode categorical variable

label_encoder_gender= LabelEncoder()

In [12]:
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [13]:
## One Hot Encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [14]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
# Combine one hot encoded columns with original data
data = pd.concat([data.drop('Geography',axis=1),pd.DataFrame((geo_encoder.toarray()),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))], axis=1)


In [17]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
# Save the encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [19]:
## Divide the data into features and target variable
X= data.drop('Exited',axis=1)
y= data['Exited']

In [20]:
## Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
## Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [23]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import datetime

In [24]:
## Build our ANN model
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],))) # 1 Hidden layer with 16 neurons
model.add(Dense(32,activation='relu')) # 2nd Hidden layer with 32 neurons
model.add(Dense(1,activation='sigmoid')) # Output layer

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
opt = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.BinaryCrossentropy()

In [27]:
## compile the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])


In [28]:
## Set up the tensorboard callback
from numpy import histogram


log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [29]:
## Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [30]:
## Train the model
history = model.fit(X_train_scaled,y_train, validation_data=(X_test_scaled,y_test),epochs=50
                    , callbacks=[tensorflow_callback,early_stopping])


Epoch 1/50
250/250 [==============================] - 2s 4ms/step - loss: 0.4040 - accuracy: 0.8305 - val_loss: 0.3493 - val_accuracy: 0.8555
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3510 - accuracy: 0.8562 - val_loss: 0.3589 - val_accuracy: 0.8605
Epoch 3/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3469 - accuracy: 0.8580 - val_loss: 0.3435 - val_accuracy: 0.8560
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3425 - accuracy: 0.8589 - val_loss: 0.3539 - val_accuracy: 0.8565
Epoch 5/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3392 - accuracy: 0.8602 - val_loss: 0.3407 - val_accuracy: 0.8620
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3381 - accuracy: 0.8594 - val_loss: 0.3370 - val_accuracy: 0.8600
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3340 - accuracy: 0.8619 - val_loss: 0.3513 - val_accuracy: 0.8560
Epoch 

In [ ]:
model.save("_model.h5")

c:\Users\NIDHI\anaconda3\envs\newenv311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# model.save('true_model.h5') # Why .h5? => .h5 is a file format used to save Keras models. 
# It stands for Hierarchical Data Format version 5. This format allows you to save the entire model architecture, weights, and training configuration in a single file, 
# making it easy to load and use the model later without needing to reconstruct it from code. 

     

In [33]:
## Load Tensorboard extension
%load_ext tensorboard

In [34]:
# %tensorboard --logdir logs/fit

In [35]:
print("Number of layers:", len(model.layers))
print("Total parameters:", model.count_params())
print("Weights arrays:", len(model.get_weights()))


Number of layers: 3
Total parameters: 2945
Weights arrays: 6


In [36]:
print(sys.executable)


c:\Users\NIDHI\anaconda3\envs\newenv311\python.exe


In [37]:
!pip install h5py


In [38]:
import tensorflow as tf
import keras
print(tf.__version__, keras.__version__)


2.13.0 2.13.1


In [39]:
import numpy
print(numpy.__version__)


1.24.3


In [40]:
import pandas
print(pandas.__version__)

2.3.3


In [42]:
import joblib

joblib.dump(label_encoder_gender, "label_encoder_gender.joblib")
joblib.dump(one_hot_encoder_geo, "onehot_encoder_geo.joblib")
joblib.dump(scaler, "scaler.joblib")


['scaler.joblib']